In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))
from pprint import pprint
from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function, LinearNormalizer, GaussianNormalizer, PercentileNormalizer
from utils.utils import get_game_name_and_scores

from models.ncf_model import NCFModel

In [2]:
# Test Fine Tuning on live data
model = NCFModel()

data_loader = DataLoader.load_from_file('test_evaluation_data_loader', use_published_models_path=True, load_live_data_loader=True)
model.load('evaluation_test_ncf_low_weight_decay_increased_lr_best_model_bugfix', load_published_model=True)
# data_loader = DataLoader.load_from_file('test_evaluation_data_loader_embed_all_except_tags_genres', use_published_models_path=True, load_live_data_loader=True)
# model.load('evaluation_test_ncf_embed_all_except_tags_genres_best_model_bugfix_clip_embeddings', load_published_model=True)

model.new_seed(None)
model.set_data_loader(data_loader)
user_to_fine_tune = 76561198103368250 # 76561198166465514 # 76561198835352289 #  #  #
print('Before', model.get_score_between_user_and_game(76561198880605436, 367520))
# model.ncf.add_new_user()
# model.user_to_index[user_to_fine_tune] = len(model.user_nodes)
# model.user_nodes.append(user_to_fine_tune)
# display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[:10]))
# display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[-10:]))
model.fine_tune(user_to_fine_tune, debug=False)
print('After', model.get_score_between_user_and_game(76561198880605436, 367520))
print('Fine Tuned User Output')
print(model.get_score_between_user_and_game(user_to_fine_tune, 367520))
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[:10]))
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[-10:]))

Before 0.6973773837089539


In [2]:
# Test cold fine tune eval on a user
data_loader = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings = ['playtime_forever'], users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], game_embeddings=['name'])
data_loader.load_random_user_train_tune_test_split(train_user_percentage=0.8, test_user_percentage=0.2, fine_tune_edge_percentage=0.8, test_edge_percentage=0.2, seed=0)

In [3]:
# Test cold fine tune eval on a user
import pandas as pd
model = NCFModel()
model.load('evaluation_cold_fine_tune_test_ncf', load_published_model=False)
model.new_seed(None)
model.set_data_loader(data_loader)
# user_to_fine_tune = 76561198021788348 # 5 tune 2 test
# user_to_fine_tune = 76561199061878606 # 18 tune 2 test
# user_to_fine_tune = 76561198255585338 # 41 tune 12 test
# user_to_fine_tune = 76561198817614482 # 49 tune 11 test
# user_to_fine_tune = 76561198202035709 # 259 tune 69 test
for user_to_fine_tune in [76561198021788348, 76561199061878606, 76561198255585338, 76561198817614482, 76561198202035709]:
    fake_interactions = pd.DataFrame(columns=model.data_loader.users_games_df.columns).astype(model.data_loader.users_games_df.dtypes)
    users_games_df = model.data_loader.users_games_df_grouped_by_user.get_group(user_to_fine_tune)
    all_users_games_df_without_test = users_games_df[users_games_df['data_split'] != 'test']
    new_users_games_df = users_games_df[users_games_df['data_split'] == 'tune']
    model._fine_tune(user_to_fine_tune, new_users_games_df, fake_interactions, all_users_games_df_without_test, fake_interactions, debug=False)
    print('Fine Tuned User Output')
    display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[:10]))
    display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(user_to_fine_tune)[-10:]))

Fine Tuned User Output


,id,name,score
0,354950,East Tower - Akio (East Tower Series Vol. 1),0.901727
1,772500,Distortions,0.877483
2,709340,Spirits of Mystery: The Dark Minotaur Collecto...,0.873220
3,589420,Grass Cutter - Mutated Lawns,0.850112
4,1247640,Dwarf Shop,0.850032
5,460780,Avadon 3: The Warborn,0.845386
6,1902710,Weird RPG,0.842512
7,1264180,Blueplanet VR v2,0.841672
8,696250,Lost Shipwreck,0.841606
9,764830,Snowmania,0.841310


,id,name,score
0,430290,Shmups Skill Test シューティング技能検定,0.411081
1,1220370,Ever Forward,0.410925
2,726250,Unknightly,0.410246
3,1406720,Dire Wolf Game Room,0.407197
4,1612770,Sweet Transit,0.406065
5,1354520,Wild Honesty: A party game for deeper conversa...,0.403916
6,27810,Gridrunner Revolution,0.399370
7,1135240,Men of War: Assault Squad 2 - Cold War,0.391639
8,1296360,Archvale,0.388162
9,678170,Slingshot Hero VR,0.382803


Fine Tuned User Output


,id,name,score
0,1141020,Adventure Escape Mysteries,1.226813
1,446480,New York Bus Simulator,1.179088
2,696250,Lost Shipwreck,1.160914
3,1665190,Monster Prom 3: Monster Roadtrip,1.153624
4,702940,Paper Shakespeare: To Date Or Not To Date?,1.138539
5,200550,Dungeons - The Dark Lord,1.137510
6,895140,JJBoom,1.128137
7,1668460,The Fancy Pants Adventures: Classic Pack,1.127601
8,2729390,JEWELiNX,1.121116
9,1575450,Dark Siren,1.109394


,id,name,score
0,2244100,Stolen Wife ~Cucked On A Hot Spring Company Trip~,0.550103
1,412480,Turtle Odyssey,0.549932
2,861230,Arevoatl Seven Coins,0.549742
3,1557350,Wanderlust: The Bermuda Secret Collector's Edi...,0.549695
4,460140,Midnight at the Celestial Palace: Part I,0.547647
5,278850,Spaceforce Homeworld,0.547645
6,454390,Rally Copters,0.547456
7,1600200,Solitaire Jack Frost Winter Adventures 2,0.546593
8,1129540,Orc Massage,0.542676
9,493000,Battle Knights,0.532711


In [2]:
# data_loader = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], user_game_edge_embeddings = ['playtime_forever'], game_embeddings=['name'])
data_loader = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings = ['playtime_forever'], users_games_edge_score_normalizers = [GaussianNormalizer(1.0, 1.0)], interactions_score_normalizers = [GaussianNormalizer(0.0, 1.0)], game_embeddings=['numReviews', 'avgReviewScore', 'price', 'numFollowers', 'name'])
data_loader.load_random_edge_train_test_split(train_percentage=0.9, test_percentage=0.1, seed=0)
data_loader.save_data_loader_parameters('test_ncf_data_loader', overwrite=True)

In [5]:
# base collaborative filter test
save_file_name = 'test_cf_model'
cf_model = NCFModel(num_epochs = 50, embedding_size = 48, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='cf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
cf_model.set_data_loader(data_loader)
cf_model.train(debug=True)
cf_model.save(save_file_name, overwrite=True)
print(cf_model.get_score_between_user_and_game(76561198835352289, 967050))
print(cf_model.get_score_between_user_and_game(76561198835352289, 578080))
print(cf_model.get_scores_between_users_and_games([76561198835352289, 76561198835352289], [967050, 578080]))
display(get_game_name_and_scores(data_loader, cf_model.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, cf_model.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

Known Game Embeddings:  ['avg_review_score', 'num_reviews', 'price', 'num_followers']
Total Learnable Parameters: 5760552


Training:   0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([2176443, 1]) torch.Size([2176443, 1])


Training:   0%|          | 0/50 [00:11<?, ?it/s]


KeyboardInterrupt: Interrupted by user

In [ ]:
# generalized collaborative filter test
save_file_name = 'test_gcf_model'
gcf_model = NCFModel(num_epochs = 0, embedding_size = 48, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='gcf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
gcf_model.set_data_loader(data_loader)
gcf_model.train(debug=True)
gcf_model.save(save_file_name, overwrite=True)
print(gcf_model.get_score_between_user_and_game(76561198835352289, 967050))
print(gcf_model.get_score_between_user_and_game(76561198835352289, 578080))
print(gcf_model.get_scores_between_users_and_games([76561198835352289, 76561198835352289], [967050, 578080]))
display(get_game_name_and_scores(data_loader, gcf_model.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, gcf_model.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

In [ ]:
# mlp test
save_file_name = 'test_mlp_model'
mlp_model = NCFModel(num_epochs = 0, embedding_size = 50, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [256, 256, 256], seed=0, model_type='mlp', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
mlp_model.set_data_loader(data_loader)
mlp_model.train(debug=True)
mlp_model.save(save_file_name, overwrite=True)
print(mlp_model.get_score_between_user_and_game(76561198835352289, 967050))
print(mlp_model.get_score_between_user_and_game(76561198835352289, 578080))
print(mlp_model.get_scores_between_users_and_games([76561198835352289, 76561198835352289], [967050, 578080]))
display(get_game_name_and_scores(data_loader, mlp_model.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, mlp_model.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

In [ ]:
# ncf test
save_file_name = 'test_ncf_model'
ncf_model = NCFModel(num_epochs = 0, embedding_size = 24, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [256, 256, 256], seed=0, model_type='ncf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
ncf_model.set_data_loader(data_loader)
ncf_model.train(debug=True)
ncf_model.save(save_file_name, overwrite=True)
print(ncf_model.get_score_between_user_and_game(76561198835352289, 967050))
print(ncf_model.get_score_between_user_and_game(76561198835352289, 578080))
print(ncf_model.get_scores_between_users_and_games([76561198835352289, 76561198835352289], [967050, 578080]))
display(get_game_name_and_scores(data_loader, ncf_model.score_and_predict_n_games_for_user(76561198835352289)[:10]))
display(get_game_name_and_scores(data_loader, ncf_model.score_and_predict_n_games_for_user(76561198835352289)[-10:]))

In [14]:
data_loader = DataLoader.load_from_file('test_evaluation_data_loader_embed_all_except_tags_genres', use_published_models_path=True, load_live_data_loader=True)
model = NCFModel()
model.load('evaluation_test_cf_embed_all_except_tags_genres_best_model_bugfix_clip_embeddings', load_published_model=True)
model.set_data_loader(data_loader)

In [15]:
# test_user = 76561199048589805 # Only 1 game
test_user = 76561198835352289 # Only 4 games
# test_user = 76561198880605436 # ~20 games
# test_user = 76561199040442492 # ~400 games
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(test_user)[:10]))
display(get_game_name_and_scores(data_loader, model.score_and_predict_n_games_for_user(test_user)[-10:]))

,id,name,score
0,294100,RimWorld,0.589256
1,779340,Total War: THREE KINGDOMS,0.541197
2,1716740,Starfield,0.518708
3,1063730,New World,0.500298
4,1144200,Ready or Not,0.497339
5,1551360,Forza Horizon 5,0.492448
6,552990,World of Warships,0.486344
7,460930,Tom Clancy's Ghost Recon® Wildlands,0.471119
8,552520,Far Cry® 5,0.467661
9,582160,Assassin's Creed® Origins,0.462494


,id,name,score
0,502150,Interstellar Logistics Inc,-0.160361
1,409870,Phantom Brave PC,-0.168625
2,2800,X2: The Threat,-0.179539
3,553820,DeathMetal,-0.179784
4,522730,God Of Arrows VR,-0.186215
5,307210,Will Fight for Food: Super Actual Sellout: Gam...,-0.193545
6,485980,Syrian Warfare,-0.197546
7,337410,Karos,-0.200550
8,463150,BARRIER X,-0.233835
9,450850,Cube Land Arena,-0.269478
